# Pipeline Analítico de Informalidad Laboral - Región de Los Ríos (Datos Limpios)

Este cuaderno documenta un flujo reproducible desde la carga de datasets ya limpios hasta la preparación de insumos analíticos (feature engineering, splits y guardado). Cada sección explica objetivos y decisiones.

## 1. Configuración del Entorno e Importación de Librerías
Objetivo: Definir librerías, silenciar warnings no críticos y ajustar opciones de visualización para inspección clara.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import json, warnings, math, datetime as dt

# Intentar importar sklearn (no obligatorio)
try:
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
    SKLEARN_AVAILABLE = True
except Exception:
    SKLEARN_AVAILABLE = False
    print('Aviso: sklearn no disponible, se omitiran pasos de encoding/escala.')

warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: f"{x:,.3f}")
pd.set_option('display.max_columns', 50)
print('Entorno listo. sklearn:', SKLEARN_AVAILABLE)

Entorno listo. sklearn: True


## 2. Definición de Rutas y Parámetros
Objetivo: Centralizar rutas y parámetros reproducibles (semilla, umbrales, carpetas de salida).

In [ ]:
BASE_DIR = Path('..').resolve()  # notebooks -> raíz proyecto
DATA_CLEAN_DIR = BASE_DIR / 'data_clean'
OUTPUT_DIR = BASE_DIR / 'data_processed_ml'
OUTPUT_DIR.mkdir(exist_ok=True)
SEED = 42
OUTLIER_Z = 3.0
CORR_THRESHOLD = 0.6
TIMESTAMP = dt.datetime.now().strftime('%Y%m%d_%H%M%S')
print('Rutas configuradas:')
print('Data limpia:', DATA_CLEAN_DIR)
print('Salida:', OUTPUT_DIR)

Rutas configuradas:
Data limpia: /Users/brunosanmartinnavarro/Documents/UACh/education-employment-analysis/data_clean
Salida: /Users/brunosanmartinnavarro/Documents/UACh/education-employment-analysis/data_processed_ml


## 3. Carga de Datasets Limpios
Objetivo: Cargar a memoria todos los archivos limpios requeridos manteniéndolos en un contenedor único para iteraciones consistentes.

In [ ]:
files_expected = [
    'INF_OI_CHL14_clean.csv',
    'INF_TOSI_CHL14_clean.csv',
    'INF_NOAGRO_CHL14_clean.csv'
 ]
datasets = {}
for fname in files_expected:
    fpath = DATA_CLEAN_DIR / fname
    if not fpath.exists():
        print(f'Archivo no encontrado: {fname}')
        continue
    df = pd.read_csv(fpath)
    datasets[fname] = df
    print(f'Cargado: {fname} -> {df.shape[0]} filas, {df.shape[1]} columnas')

list(datasets.keys())

Cargado: INF_OI_CHL14_clean.csv -> 327 filas, 11 columnas
Cargado: INF_TOSI_CHL14_clean.csv -> 327 filas, 11 columnas
Cargado: INF_NOAGRO_CHL14_clean.csv -> 327 filas, 11 columnas


['INF_OI_CHL14_clean.csv',
 'INF_TOSI_CHL14_clean.csv',
 'INF_NOAGRO_CHL14_clean.csv']

## 4. Inspección Rápida de Estructura
Objetivo: Confirmar estructura, tipos y carga efectiva.

In [ ]:
for name, df in datasets.items():
    # Encabezado claro por dataset
    print(f"=== {name} ===")
    print('Shape:', df.shape)
    print('Columnas:', list(df.columns))
    print('Tipos:')
    print(df.dtypes)
    print('Memoria (KB):', round(df.memory_usage(deep=True).sum()/1024,2))
    display(df.head(3))

=== INF_OI_CHL14_clean.csv ===
Shape: (327, 11)
Columnas: ['DTI_CL_INDICADOR', 'Indicador', 'DTI_CL_TRIMESTRE_MOVIL', 'Trimestre Móvil', 'DTI_CL_REGION', 'Región', 'DTI_CL_SEXO', 'Sexo', 'Value', 'Flag Codes', 'Flags']
Tipos:
DTI_CL_INDICADOR           object
Indicador                  object
DTI_CL_TRIMESTRE_MOVIL     object
Trimestre Móvil            object
DTI_CL_REGION              object
Región                     object
DTI_CL_SEXO                object
Sexo                       object
Value                     float64
Flag Codes                float64
Flags                     float64
dtype: object
Memoria (KB): 173.89


,DTI_CL_INDICADOR,Indicador,DTI_CL_TRIMESTRE_MOVIL,Trimestre Móvil,DTI_CL_REGION,Región,DTI_CL_SEXO,Sexo,Value,Flag Codes,Flags
0,INF_OCU_INF,Ocupados informales (proyecciones base 2002),2017-V08,2017 jul-sep,CHL14,Región de Los Ríos,_T,Ambos sexos,70.213,NaN,NaN
1,INF_OCU_INF,Ocupados informales (proyecciones base 2002),2017-V08,2017 jul-sep,CHL14,Región de Los Ríos,M,Hombres,41.440,NaN,NaN
2,INF_OCU_INF,Ocupados informales (proyecciones base 2002),2017-V08,2017 jul-sep,CHL14,Región de Los Ríos,F,Mujeres,28.773,NaN,NaN


=== INF_TOSI_CHL14_clean.csv ===
Shape: (327, 11)
Columnas: ['DTI_CL_INDICADOR', 'Indicador', 'DTI_CL_TRIMESTRE_MOVIL', 'Trimestre Móvil', 'DTI_CL_REGION', 'Región', 'DTI_CL_SEXO', 'Sexo', 'Value', 'Flag Codes', 'Flags']
Tipos:
DTI_CL_INDICADOR           object
Indicador                  object
DTI_CL_TRIMESTRE_MOVIL     object
Trimestre Móvil            object
DTI_CL_REGION              object
Región                     object
DTI_CL_SEXO                object
Sexo                       object
Value                     float64
Flag Codes                float64
Flags                     float64
dtype: object
Memoria (KB): 184.51


,DTI_CL_INDICADOR,Indicador,DTI_CL_TRIMESTRE_MOVIL,Trimestre Móvil,DTI_CL_REGION,Región,DTI_CL_SEXO,Sexo,Value,Flag Codes,Flags
0,INF_TOSI,Tasa de ocupación en el sector informal (proy...,2017-V08,2017 jul-sep,CHL14,Región de Los Ríos,_T,Ambos sexos,22.405,NaN,NaN
1,INF_TOSI,Tasa de ocupación en el sector informal (proy...,2017-V08,2017 jul-sep,CHL14,Región de Los Ríos,M,Hombres,24.411,NaN,NaN
2,INF_TOSI,Tasa de ocupación en el sector informal (proy...,2017-V08,2017 jul-sep,CHL14,Región de Los Ríos,F,Mujeres,19.438,NaN,NaN


=== INF_NOAGRO_CHL14_clean.csv ===
Shape: (327, 11)
Columnas: ['DTI_CL_INDICADOR', 'Indicador', 'DTI_CL_TRIMESTRE_MOVIL', 'Trimestre Móvil', 'DTI_CL_REGION', 'Región', 'DTI_CL_SEXO', 'Sexo', 'Value', 'Flag Codes', 'Flags']
Tipos:
DTI_CL_INDICADOR           object
Indicador                  object
DTI_CL_TRIMESTRE_MOVIL     object
Trimestre Móvil            object
DTI_CL_REGION              object
Región                     object
DTI_CL_SEXO                object
Sexo                       object
Value                     float64
Flag Codes                float64
Flags                     float64
dtype: object
Memoria (KB): 193.05


,DTI_CL_INDICADOR,Indicador,DTI_CL_TRIMESTRE_MOVIL,Trimestre Móvil,DTI_CL_REGION,Región,DTI_CL_SEXO,Sexo,Value,Flag Codes,Flags
0,INF_TOI_NOAGRO,Tasa de ocupación informal excluyendo al secto...,2017-V08,2017 jul-sep,CHL14,Región de Los Ríos,_T,Ambos sexos,34.646,NaN,NaN
1,INF_TOI_NOAGRO,Tasa de ocupación informal excluyendo al secto...,2017-V08,2017 jul-sep,CHL14,Región de Los Ríos,M,Hombres,33.766,NaN,NaN
2,INF_TOI_NOAGRO,Tasa de ocupación informal excluyendo al secto...,2017-V08,2017 jul-sep,CHL14,Región de Los Ríos,F,Mujeres,35.789,NaN,NaN


## 5. Verificación de Calidad Post-Limpieza
Objetivo: Validar ausencia de problemas residuales (nulos, duplicados, claves inconsistentes).

In [ ]:
quality_reports = []
for name, df in datasets.items():
    null_pct = (df.isna().mean()*100).round(2)
    duplicates = df.duplicated().sum()
    row_example = df.iloc[0].to_dict() if not df.empty else {}
    keys = [c for c in ['Trimestre Móvil','DTI_CL_SEXO'] if c in df.columns]
    uniques = {k: df[k].nunique() for k in keys}
    quality_reports.append({
        'dataset': name,
        'filas': len(df),
        'duplicados': duplicates,
        'max_null_pct': null_pct.max() if len(null_pct)>0 else 0,
        'claves': uniques
    })
    print(f"{name}: filas={len(df)}, duplicados={duplicates}, max_null%={null_pct.max() if len(null_pct)>0 else 0}")

pd.DataFrame(quality_reports)

INF_OI_CHL14_clean.csv: filas=327, duplicados=0, max_null%=100.0
INF_TOSI_CHL14_clean.csv: filas=327, duplicados=0, max_null%=100.0
INF_NOAGRO_CHL14_clean.csv: filas=327, duplicados=0, max_null%=100.0


,dataset,filas,duplicados,max_null_pct,claves
0,INF_OI_CHL14_clean.csv,327,0,100.000,"{'Trimestre Móvil': 81, 'DTI_CL_SEXO': 3}"
1,INF_TOSI_CHL14_clean.csv,327,0,100.000,"{'Trimestre Móvil': 81, 'DTI_CL_SEXO': 3}"
2,INF_NOAGRO_CHL14_clean.csv,327,0,100.000,"{'Trimestre Móvil': 81, 'DTI_CL_SEXO': 3}"


## 6. Diccionario de Variables (Generado)
Objetivo: Construir un resumen estructurado de cada columna para documentación reproducible.

In [ ]:
diccionario_list = []
for name, df in datasets.items():
    for col in df.columns:
        serie = df[col]
        diccionario_list.append({
            'dataset': name,
            'columna': col,
            'dtype': str(serie.dtype),
            'n_null': serie.isna().sum(),
            '%null': round(serie.isna().mean()*100,2),
            'n_unique': serie.nunique(),
            'ejemplo': serie.dropna().iloc[0] if serie.dropna().shape[0]>0 else None
        })

diccionario_df = pd.DataFrame(diccionario_list)
diccionario_path = OUTPUT_DIR / f'diccionario_variables_{TIMESTAMP}.csv'
diccionario_df.to_csv(diccionario_path, index=False)
print('Diccionario exportado:', diccionario_path)
display(diccionario_df.head())

Diccionario exportado: /Users/brunosanmartinnavarro/Documents/UACh/education-employment-analysis/data_processed_ml/diccionario_variables_20251113_025426.csv


,dataset,columna,dtype,n_null,%null,n_unique,ejemplo
0,INF_OI_CHL14_clean.csv,DTI_CL_INDICADOR,object,0,0.000,2,INF_OCU_INF
1,INF_OI_CHL14_clean.csv,Indicador,object,0,0.000,2,Ocupados informales (proyecciones base 2002)
2,INF_OI_CHL14_clean.csv,DTI_CL_TRIMESTRE_MOVIL,object,0,0.000,81,2017-V08
3,INF_OI_CHL14_clean.csv,Trimestre Móvil,object,0,0.000,81,2017 jul-sep
4,INF_OI_CHL14_clean.csv,DTI_CL_REGION,object,0,0.000,1,CHL14


## 7. Unificación / Merge de Datasets
Objetivo: Construir una tabla integrada alineada por trimestre y sexo para facilitar análisis longitudinal y comparativo.

## 9.1 Visualizaciones Temporales Estilo The Economist
Objetivo: Representar la evolución de los indicadores clave con un estilo editorial limpio:

- Tipografía: Georgia, serif
- Sin grillas verticales
- Grillas horizontales suaves
- Línea COVID-19 (primer trimestre 2020 disponible)
- Línea de promedio como referencia contextual

Se generan tres gráficos: (1) Tasa informal, (2) Ocupados informales (personas), (3) Tasa informal no agro. 

In [ ]:
# Integrar indicadores claves en un solo DataFrame ordenado
rename_map = {
    'INF_TOSI_CHL14_clean.csv': 'tasa_informal_sector',
    'INF_OI_CHL14_clean.csv': 'ocupados_informales',
    'INF_NOAGRO_CHL14_clean.csv': 'tasa_informal_noagro'
}
renamed_min = {}
for fname, df in datasets.items():
    temp = df.copy()
    if 'Value' in temp.columns:
        temp = temp.rename(columns={'Value': rename_map[fname]})
    keep_cols = ['Trimestre Móvil', 'DTI_CL_SEXO', rename_map[fname]]
    renamed_min[fname] = temp[keep_cols]

merged = None
for fname, df in renamed_min.items():
    if merged is None:
        merged = df.copy()
    else:
        merged = merged.merge(df, on=['Trimestre Móvil', 'DTI_CL_SEXO'], how='outer')

if merged[['Trimestre Móvil', 'DTI_CL_SEXO']].duplicated().any():
    agg_map = {c: 'mean' for c in merged.columns if c not in ['Trimestre Móvil', 'DTI_CL_SEXO']}
    merged = merged.groupby(['Trimestre Móvil', 'DTI_CL_SEXO'], as_index=False).agg(agg_map)

merged = merged.sort_values(['Trimestre Móvil', 'DTI_CL_SEXO']).reset_index(drop=True)
print('Shape merged:', merged.shape)
print('Columnas:', list(merged.columns))
display(merged.head())

Shape merged: (243, 5)
Columnas: ['Trimestre Móvil', 'DTI_CL_SEXO', 'ocupados_informales', 'tasa_informal_sector', 'tasa_informal_noagro']


,Trimestre Móvil,DTI_CL_SEXO,ocupados_informales,tasa_informal_sector,tasa_informal_noagro
0,2017 ago-oct,F,29.610,20.068,37.607
1,2017 ago-oct,M,40.899,24.949,33.888
2,2017 ago-oct,_T,70.508,22.994,35.497
3,2017 jul-sep,F,28.803,18.815,35.973
4,2017 jul-sep,M,41.074,23.562,33.761


In [ ]:
# Configuración de Visualizaciones - Estilo The Economist
import plotly.graph_objects as go
import plotly.express as px

# Helper estilo (se mantiene)
STYLE = dict(font=dict(family='Georgia, serif', color='#1e293b'),
             title=dict(font=dict(size=18, family='Georgia, serif', color='#1e293b')),
             legend=dict(font=dict(family='Georgia, serif')))

axis_style = dict(
    title=dict(font=dict(family='Georgia, serif')),
    tickfont=dict(family='Georgia, serif'),
    linecolor='#cbd5e1',
    showgrid=False,
    zeroline=False
)

axis_style_y = dict(
    title=dict(font=dict(family='Georgia, serif')),
    tickfont=dict(family='Georgia, serif'),
    linecolor='#cbd5e1',
    gridcolor='#e2e8f0',
    gridwidth=0.6,
    showgrid=True,
    zeroline=False
)

# Asegurar merge
if 'merged' not in globals():
    raise RuntimeError('Ejecuta las celdas previas hasta el merge antes de generar gráficos.')

# Línea COVID (mantiene periodos categóricos)
def add_covid(fig, df):
    covid_trimestres = df[df['Trimestre Móvil'].str.contains('2020', na=False)]['Trimestre Móvil'].unique()
    if len(covid_trimestres) == 0:
        return fig
    t0 = sorted(covid_trimestres)[0]
    fig.add_shape(type='line', x0=t0, x1=t0, y0=0, y1=1, yref='paper', line=dict(color='#e11d48', width=2, dash='dash'))
    fig.add_annotation(x=t0, y=0.95, yref='paper', text='<b>COVID-19</b>', showarrow=True, arrowhead=2,
                       arrowcolor='#e11d48', ax=0, ay=-28,
                       font=dict(family='Georgia, serif', size=10, color='#e11d48'),
                       bgcolor='rgba(255,255,255,0.85)', bordercolor='#e11d48', borderwidth=1)
    return fig

# Base ordenada por periodo (categórico)
serie_base = merged.sort_values('Trimestre Móvil')

print('Configuracion de visualizaciones lista')

Configuracion de visualizaciones lista


### 9.2 📊 Visualización 1: Tasa de Ocupación Informal (%)

**¿Qué mide este indicador?**
- Porcentaje de trabajadores ocupados en empleos informales respecto del total de ocupados
- Refleja la magnitud relativa de la informalidad laboral en la economía
- Indicador clave para evaluar la calidad del empleo

**Interpretación:**
- **Valores en porcentaje**: Expresado como porcentaje del total de ocupados
- **Tendencia temporal**: Permite observar evoluciones de la formalización laboral
- **Impacto COVID-19**: Marca la línea temporal del primer trimestre 2020
- **Línea de promedio**: Referencia histórica para contextualizar períodos específicos

**Color identificativo:** 🔴 Rojo (#dc2626)

In [ ]:
# Visualización 1: Tasa de ocupación informal (%)
col = 'tasa_informal_sector'
label = 'Tasa de ocupación informal (%)'
highlight_color = '#d62828'
neutral_color = '#0a9396'
background_color = '#f7f2ea'
grid_color = '#d9d3c5'
text_color = '#1e1b18'
spike_color = 'rgba(120, 120, 120, 0.35)'

if col not in serie_base.columns:
    print(f'Omitido {col} (no existe)')
else:
    from unicodedata import normalize
    import math

    df_plot = serie_base[['Trimestre Móvil', 'DTI_CL_SEXO', col]].copy()
    if '_T' in df_plot['DTI_CL_SEXO'].unique():
        df_plot = df_plot[df_plot['DTI_CL_SEXO'] == '_T']
    df_plot = df_plot.drop_duplicates(subset=['Trimestre Móvil']).sort_values('Trimestre Móvil')

    month_map = {
        'ENE': 1, 'FEB': 2, 'MAR': 3, 'ABR': 4, 'MAY': 5, 'JUN': 6,
        'JUL': 7, 'AGO': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DIC': 12
    }

    def periodo_to_timestamp(periodo):
        if not isinstance(periodo, str):
            return pd.NaT
        parts = periodo.split(' ', 1)
        if len(parts) != 2:
            return pd.NaT
        year, meses = parts
        primer_mes = meses.split('-')[0].strip()
        clave = normalize('NFKD', primer_mes).encode('ascii', 'ignore').decode('ascii').upper()[:3]
        month = month_map.get(clave)
        if month is None:
            return pd.NaT
        return pd.Timestamp(int(year), month, 1)

    df_plot['periodo_dt'] = df_plot['Trimestre Móvil'].apply(periodo_to_timestamp)
    df_plot = df_plot.dropna(subset=['periodo_dt'])
    df_plot = df_plot.sort_values('periodo_dt')

    if df_plot.empty:
        print('Sin datos para linea temporal')
    else:
        font_family = globals().get('ECONOMIST_FONT', 'Georgia, serif')
        df_plot['label'] = df_plot[col].apply(lambda v: f"{v:,.1f}".replace(',', 'X').replace('.', ',').replace('X', '.'))

        covid_start = df_plot[df_plot['periodo_dt'].dt.year == 2020]
        covid_marker = covid_start['periodo_dt'].iloc[0] if not covid_start.empty else None

        tooltip_template = (
            "<span style='background-color:#ffffff;border:1px solid #ffffff;"
            "padding:6px 10px;display:inline-block;color:#1e1b18;'>"
            "<b>%{x|%Y %b}</b><br>"
            "Tasa de ocupación informal (%): %{y:.1f}%"
            "</span><extra></extra>"
        )

        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                x=df_plot['periodo_dt'],
                y=df_plot[col],
                mode='lines+markers',
                line=dict(color=neutral_color, width=3),
                marker=dict(color=neutral_color, size=8, line=dict(color='#ffffff', width=1.2)),
                hovertemplate=tooltip_template,
                showlegend=False
            )
        )

        if highlight_color and not df_plot.empty:
            max_row = df_plot.loc[df_plot[col].idxmax()]
            fig.add_trace(
                go.Scatter(
                    x=[max_row['periodo_dt']],
                    y=[max_row[col]],
                    mode='markers',
                    marker=dict(color=highlight_color, size=12, symbol='diamond', line=dict(color='#ffffff', width=1.5)),
                    hovertemplate='<b>Pico</b><br>'+label+': %{y:.1f}%<extra></extra>',
                    showlegend=False
                )
            )

        if covid_marker is not None:
            fig.add_vline(x=covid_marker, line=dict(color='#e11d48', width=2, dash='dash'))
            fig.add_annotation(
                x=covid_marker,
                y=df_plot[col].max(),
                text='<b>COVID-19</b>',
                showarrow=False,
                font=dict(family=font_family, size=10, color='#e11d48'),
                bgcolor='rgba(255,255,255,0.85)',
                bordercolor='#e11d48',
                borderwidth=1
            )

        fig.update_layout(
            template='plotly_white',
            title=dict(
                text='<b>Línea temporal: Tasa de ocupación informal (2017-2024)</b>',
                font=dict(family=font_family, size=24, color=text_color),
                x=0.05,
                y=0.95
            ),
            font=dict(family=font_family, size=15, color=text_color),
            plot_bgcolor=background_color,
            paper_bgcolor=background_color,
            margin=dict(l=120, r=120, t=110, b=100),
            hoverlabel=dict(bgcolor='rgba(0,0,0,0)', bordercolor='rgba(0,0,0,0)', font=dict(family=font_family, color=text_color)),
            hovermode='x',
            spikedistance=-1,
            hoverdistance=50,
            width=1200,
            height=620
        )

        fig.update_xaxes(
            title=dict(text='Trimestre móvil', font=dict(family=font_family, size=14, color=text_color)),
            showgrid=True,
            gridcolor=grid_color,
            griddash='dot',
            zeroline=False,
            tickformat='%Y',
            tickfont=dict(family=font_family, size=12, color=text_color),
            showspikes=True,
            spikemode='across',
            spikecolor=spike_color,
            spikethickness=1.2,
            spikedash='solid',
            spikesnap='cursor',
            hoverformat=' '
        )
        fig.update_yaxes(
            title=dict(text='Porcentaje', font=dict(family=font_family, size=14, color=text_color)),
            showgrid=True,
            gridcolor=grid_color,
            griddash='dot',
            zeroline=False,
            tickfont=dict(family=font_family, size=12, color=text_color),
            showspikes=False
        )

        fig.update_layout(
            annotations=[
                dict(
                    text='Los Ríos, ENE 2017 - MAY 2024',
                    x=0,
                    y=1.05,
                    xref='paper',
                    yref='paper',
                    showarrow=False,
                    font=dict(family=font_family,
                     size=14, color='#4a4338')
                ),
                dict(
                    text='Fuente: Encuesta Nacional de Empleo (INE)',
                    x=0,
                    y=-0.18,
                    xref='paper',
                    yref='paper',
                    showarrow=False,
                    font=dict(family=font_family, size=11, color='#6e6252')
                )
            ]
        )

        fig.show()
        print(f'Línea temporal generada: {label}')

Línea temporal generada: Tasa de ocupación informal (%)


### 9.3 👥 Visualización 2: Ocupados Informales (personas)

**¿Qué mide este indicador?**
- Número absoluto de personas ocupadas en empleos informales
- Muestra el volumen real de trabajadores en situación de informalidad
- Permite dimensionar el impacto demográfico de las políticas laborales

**Interpretación:**
- **Valores absolutos**: Expresado en cantidad de personas (ej: 60,000 personas)
- **Tendencia demográfica**: Refleja cambios en el volumen de empleo informal
- **Variaciones estacionales**: Pueden reflejar ciclos económicos y cambios estructurales
- **Impacto pandemia**: Esperamos ver variaciones significativas en períodos de crisis

**Color identificativo:** 🔵 Azul (#2563eb)

In [ ]:
# Visualización 2: Ocupados informales (personas)
col = 'ocupados_informales'
label = 'Ocupados informales (personas)'
highlight_color = '#d62828'
neutral_color = '#0a9396'
background_color = '#f7f2ea'
grid_color = '#d9d3c5'
text_color = '#1e1b18'

if col not in serie_base.columns:
    print(f'Omitido {col} (no existe)')
else:
    from unicodedata import normalize
    import math

    df_plot = serie_base[['Trimestre Móvil', 'DTI_CL_SEXO', col]].copy()
    if '_T' in df_plot['DTI_CL_SEXO'].unique():
        df_plot = df_plot[df_plot['DTI_CL_SEXO'] == '_T']
    df_plot = df_plot.drop_duplicates(subset=['Trimestre Móvil'])

    month_map = {
        'ENE': 1, 'FEB': 2, 'MAR': 3, 'ABR': 4, 'MAY': 5, 'JUN': 6,
        'JUL': 7, 'AGO': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DIC': 12
    }

    def periodo_to_timestamp(periodo):
        if not isinstance(periodo, str):
            return pd.NaT
        parts = periodo.split(' ', 1)
        if len(parts) != 2:
            return pd.NaT
        year, meses = parts
        primer_mes = meses.split('-')[0].strip()
        clave = normalize('NFKD', primer_mes).encode('ascii', 'ignore').decode('ascii').upper()[:3]
        month = month_map.get(clave)
        if month is None:
            return pd.NaT
        return pd.Timestamp(int(year), month, 1)

    def format_spanish(number: float, decimals: int = 0) -> str:
        formatted = f"{number:,.{decimals}f}"
        return formatted.replace(',', 'X').replace('.', ',').replace('X', '.')

    df_plot['periodo_dt'] = df_plot['Trimestre Móvil'].apply(periodo_to_timestamp)
    df_plot = df_plot.dropna(subset=['periodo_dt'])
    df_plot['year'] = df_plot['periodo_dt'].dt.year
    df_year = (
        df_plot.groupby('year', as_index=False)[col]
               .mean()
               .sort_values('year')
    )
    df_plot = df_plot.drop(columns='periodo_dt')

    if df_year.empty:
        print('Sin datos para generar grafico anual')
    else:
        font_family = globals().get('ECONOMIST_FONT', 'Georgia, serif')
        df_year['value'] = df_year[col]
        df_year['label'] = df_year['value'].apply(lambda v: f"{format_spanish(v)} personas")
        highlight_year = df_year.loc[df_year['value'].idxmax(), 'year']
        df_year['color'] = df_year['year'].apply(lambda y: highlight_color if y == highlight_year else neutral_color)
        df_year['year_str'] = df_year['year'].astype(str)

        max_value = df_year['value'].max()
        x_padding = max_value * 0.12 if max_value else 5_000

        fig = go.Figure()
        fig.add_trace(
            go.Bar(
                x=df_year['value'],
                y=df_year['year_str'],
                orientation='h',
                marker=dict(color=df_year['color'], line=dict(color='#ffffff', width=1.2)),
                hovertemplate='<b>%{customdata[0]}</b><br>'+label+': %{customdata[1]:,.0f}<extra></extra>',
                customdata=list(zip(df_year['year_str'], df_year['value'])),
                text=df_year['label'],
                textposition='outside',
                textfont=dict(family=font_family, size=12, color=text_color)
            )
        )

        underline_shape = dict(
            type='line',
            x0=-0.38,
            x1=0.95,
            y0=1.08,
            y1=1.08,
            xref='paper',
            yref='paper',
            line=dict(color=highlight_color, width=4)
        )

        baseline_shape = dict(
            type='line',
            x0=0,
            x1=0,
            y0=-0.08,
            y1=1.03,
            xref='x',
            yref='paper',
            line=dict(color='#c9c1b4', width=1.2)
        )

        xaxis_max = max_value + x_padding if max_value else 1
        tick_count = 6
        tick_step = max(xaxis_max / tick_count, 10_000)
        tick_values = list(range(0, int(xaxis_max) + int(tick_step), int(tick_step)))
        tick_text = [format_spanish(val) for val in tick_values]

        fig.update_layout(
            template='plotly_white',
            title=dict(
                text='<b>Ocupados Informales en Los Ríos (2017-2024)</b>',
                font=dict(family=font_family, size=24, color=text_color),
                x=0.05,
                y=0.96
            ),
            font=dict(family=font_family, size=15, color=text_color),
            plot_bgcolor=background_color,
            paper_bgcolor=background_color,
            margin=dict(l=240, r=160, t=120, b=80),
            hoverlabel=dict(bgcolor='#ffffff', font=dict(family=font_family, color=text_color)),
            width=1200,
            height=700,
            shapes=[baseline_shape, underline_shape],
            showlegend=False
        )

        fig.update_layout(
            xaxis=dict(
                title=dict(text='Personas (promedio anual)', font=dict(family=font_family, size=14, color=text_color)),
                showgrid=True,
                gridcolor=grid_color,
                griddash='dot',
                zeroline=False,
                tickfont=dict(family=font_family, size=12, color=text_color),
                tickvals=tick_values,
                ticktext=tick_text,
                range=[0, xaxis_max]
            ),
            yaxis=dict(
                title='',
                tickfont=dict(family=font_family, size=12, color=text_color),
                showgrid=False,
                zeroline=False,
                categoryorder='array',
                categoryarray=df_year['year_str'].tolist()
            )
        )

        fig.update_layout(
            annotations=[
                dict(
                    text='Los Ríos, ENE 2017 - MAY 2024',
                    x=0,
                    y=1.04,
                    xref='paper',
                    yref='paper',
                    showarrow=False,
                    font=dict(family=font_family, size=14, color='#4a4338')
                ),
                dict(
                    text='Fuente: Encuesta Nacional de Empleo (INE)',
                    x=0,
                    y=-0.18,
                    xref='paper',
                    yref='paper',
                    showarrow=False,
                    font=dict(family=font_family, size=11, color='#6e6252')
                )
            ]
        )

        fig.update_traces(hoverlabel=dict(bgcolor='#ffffff'))
        fig.show()
        print(f'Grafico generado: {label} - {len(df_year)} anos')
        print(f'Anios incluidos: {df_year["year"].astype(str).tolist()}')

Grafico generado: Ocupados informales (personas) - 8 anos
Anios incluidos: ['2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']


### 9.4 🌾 Visualización 3: Tasa Informal No Agropecuario (%)

**¿Qué mide este indicador?**
- Porcentaje de ocupación informal excluyendo el sector agropecuario
- Ofrece una vista más urbana/industrial de la informalidad laboral
- Elimina el sesgo del sector agrícola que tradicionalmente tiene mayor informalidad

**Interpretación:**
- **Exclusión sectorial**: No incluye trabajos agrícolas, ganaderos o forestales
- **Focus urbano**: Refleja principalmente la informalidad en sectores secundarios y terciarios
- **Comparabilidad**: Permite comparar con otros países que excluyen el sector primario
- **Políticas focalizadas**: Útil para políticas dirigidas a sectores no agrícolas

**Color identificativo:** 🟢 Verde (#16a34a)

In [70]:
# Visualización 3: Tasa informal no agro (%)
col = 'tasa_informal_noagro'
label = 'Tasa informal no agro (%)'
highlight_color = '#d62828'
neutral_color = '#0a9396'
background_color = '#f7f2ea'
grid_color = '#d9d3c5'
text_color = '#1e1b18'

if col not in serie_base.columns:
    print(f'Omitido {col} (no existe)')
else:
    from unicodedata import normalize
    import math

    df_plot = serie_base[['Trimestre Móvil', 'DTI_CL_SEXO', col]].copy()
    if '_T' in df_plot['DTI_CL_SEXO'].unique():
        df_plot = df_plot[df_plot['DTI_CL_SEXO'] == '_T']
    df_plot = df_plot.drop_duplicates(subset=['Trimestre Móvil'])

    month_map = {
        'ENE': 1, 'FEB': 2, 'MAR': 3, 'ABR': 4, 'MAY': 5, 'JUN': 6,
        'JUL': 7, 'AGO': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DIC': 12
    }

    def periodo_to_timestamp(periodo):
        if not isinstance(periodo, str):
            return pd.NaT
        parts = periodo.split(' ', 1)
        if len(parts) != 2:
            return pd.NaT
        year, meses = parts
        primer_mes = meses.split('-')[0].strip()
        clave = normalize('NFKD', primer_mes).encode('ascii', 'ignore').decode('ascii').upper()[:3]
        month = month_map.get(clave)
        if month is None:
            return pd.NaT
        return pd.Timestamp(int(year), month, 1)

    def format_spanish(number: float, decimals: int = 1) -> str:
        formatted = f"{number:,.{decimals}f}"
        return formatted.replace(',', 'X').replace('.', ',').replace('X', '.')

    df_plot['periodo_dt'] = df_plot['Trimestre Móvil'].apply(periodo_to_timestamp)
    df_plot = df_plot.dropna(subset=['periodo_dt'])
    df_plot['year'] = df_plot['periodo_dt'].dt.year
    df_year = (
        df_plot.groupby('year', as_index=False)[col]
               .mean()
               .sort_values('year')
    )
    df_plot = df_plot.drop(columns='periodo_dt')

    if df_year.empty:
        print('Sin datos para generar grafico anual')
    else:
        font_family = globals().get('ECONOMIST_FONT', 'Georgia, serif')
        df_year['value'] = df_year[col]
        df_year['label'] = df_year['value'].apply(lambda v: f"{format_spanish(v)} %")
        highlight_year = df_year.loc[df_year['value'].idxmax(), 'year']
        df_year['color'] = df_year['year'].apply(lambda y: highlight_color if y == highlight_year else neutral_color)
        df_year['year_str'] = df_year['year'].astype(str)

        max_value = df_year['value'].max()
        x_padding = max_value * 0.12 if max_value else 1
        label_offset = max(max_value * 0.12, 0.6)

        fig = go.Figure()
        for _, row in df_year.iterrows():
            color = row['color']
            label_color = highlight_color if row['year'] == highlight_year else text_color
            fig.add_trace(
                go.Scatter(
                    x=[0, row['value']],
                    y=[row['year_str'], row['year_str']],
                    mode='lines',
                    line=dict(color=color, width=2.4),
                    hoverinfo='skip',
                    showlegend=False
                )
            )
            fig.add_trace(
                go.Scatter(
                    x=[row['value']],
                    y=[row['year_str']],
                    mode='markers',
                    marker=dict(color=color, size=12, line=dict(color='#ffffff', width=1.2)),
                    hovertemplate='<b>%{customdata[0]}</b><br>'+label+': %{customdata[1]:.1f}%<extra></extra>',
                    customdata=[[row['year_str'], row['value']]],
                    cliponaxis=False,
                    showlegend=False
                )
            )
            fig.add_trace(
                go.Scatter(
                    x=[row['value'] + label_offset],
                    y=[row['year_str']],
                    mode='text',
                    text=[row['label']],
                    textposition='middle left',
                    textfont=dict(family=font_family, size=12, color=label_color),
                    hoverinfo='skip',
                    showlegend=False,
                    cliponaxis=False
                )
            )

        underline_shape = dict(
            type='line',
            x0=-0.40,
            x1=0.95,
            y0=1.10,
            y1=1.10,
            xref='paper',
            yref='paper',
            line=dict(color=highlight_color, width=4)
        )

        baseline_shape = dict(
            type='line',
            x0=0,
            x1=0,
            y0=-0.08,
            y1=1.05,
            xref='x',
            yref='paper',
            line=dict(color='#c9c1b4', width=1.2)
        )

        xaxis_range = [0, max_value + x_padding + label_offset + (0.1 * max_value if max_value else 0)]
        tick_step = max_value / 6 if max_value else 0.5
        tick_step = max(tick_step, 0.5)
        tick_values = []
        current = 0.0
        while current <= xaxis_range[1] + (tick_step / 2):
            tick_values.append(round(current, 1))
            current += tick_step
        tick_text = [f"{format_spanish(val)} %" for val in tick_values]

        fig.update_layout(
            template='plotly_white',
            title=dict(
                text='<b>Tasa Informal No Agropecuario en Los Ríos (2017-2024)</b>',
                font=dict(family=font_family, size=24, color=text_color),
                x=0.05,
                y=0.96
            ),
            font=dict(family=font_family, size=15, color=text_color),
            plot_bgcolor=background_color,
            paper_bgcolor=background_color,
            margin=dict(l=250, r=220, t=120, b=80),
            hoverlabel=dict(bgcolor='#ffffff', font=dict(family=font_family, color=text_color)),
            width=1200,
            height=700,
            shapes=[baseline_shape, underline_shape],
            showlegend=False
        )

        fig.update_layout(
            xaxis=dict(
                title=dict(text='Porcentaje (promedio anual)', font=dict(family=font_family, size=14, color=text_color)),
                showgrid=True,
                gridcolor=grid_color,
                griddash='dot',
                zeroline=False,
                tickfont=dict(family=font_family, size=12, color=text_color),
                tickvals=tick_values,
                ticktext=tick_text,
                range=xaxis_range
            ),
            yaxis=dict(
                title='',
                tickfont=dict(family=font_family, size=12, color=text_color),
                showgrid=False,
                zeroline=False,
                categoryorder='array',
                categoryarray=df_year['year_str'].tolist()
            )
        )

        fig.update_layout(
            annotations=[
                dict(
                    text='Los Ríos, ENE 2017 - MAY 2024',
                    x=0,
                    y=1.04,
                    xref='paper',
                    yref='paper',
                    showarrow=False,
                    font=dict(family=font_family, size=14, color='#4a4338')
                ),
                dict(
                    text='Fuente: Encuesta Nacional de Empleo (INE)',
                    x=0,
                    y=-0.18,
                    xref='paper',
                    yref='paper',
                    showarrow=False,
                    font=dict(family=font_family, size=11, color='#6e6252')
                )
            ]
        )

        fig.show()
        print(f'Grafico generado: {label} - {len(df_year)} anos')
        print(f'Anios incluidos: {df_year["year"].astype(str).tolist()}')

Grafico generado: Tasa informal no agro (%) - 8 anos
Anios incluidos: ['2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']


In [67]:
# Visualización 3 bis: Línea temporal tasa informal no agro (%)
col = 'tasa_informal_noagro'
label = 'Tasa informal no agro (%)'
highlight_color = '#d62828'
neutral_color = '#0a9396'
background_color = '#f7f2ea'
grid_color = '#d9d3c5'
text_color = '#1e1b18'
spike_color = 'rgba(120, 120, 120, 0.35)'

if col not in serie_base.columns:
    print(f'Omitido {col} (no existe)')
else:
    from unicodedata import normalize
    import math

    df_plot = serie_base[['Trimestre Móvil', 'DTI_CL_SEXO', col]].copy()
    if '_T' in df_plot['DTI_CL_SEXO'].unique():
        df_plot = df_plot[df_plot['DTI_CL_SEXO'] == '_T']
    df_plot = df_plot.drop_duplicates(subset=['Trimestre Móvil']).sort_values('Trimestre Móvil')

    month_map = {
        'ENE': 1, 'FEB': 2, 'MAR': 3, 'ABR': 4, 'MAY': 5, 'JUN': 6,
        'JUL': 7, 'AGO': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DIC': 12
    }

    def periodo_to_timestamp(periodo):
        if not isinstance(periodo, str):
            return pd.NaT
        parts = periodo.split(' ', 1)
        if len(parts) != 2:
            return pd.NaT
        year, meses = parts
        primer_mes = meses.split('-')[0].strip()
        clave = normalize('NFKD', primer_mes).encode('ascii', 'ignore').decode('ascii').upper()[:3]
        month = month_map.get(clave)
        if month is None:
            return pd.NaT
        return pd.Timestamp(int(year), month, 1)

    df_plot['periodo_dt'] = df_plot['Trimestre Móvil'].apply(periodo_to_timestamp)
    df_plot = df_plot.dropna(subset=['periodo_dt'])
    df_plot = df_plot.sort_values('periodo_dt')

    if df_plot.empty:
        print('Sin datos para linea temporal')
    else:
        font_family = globals().get('ECONOMIST_FONT', 'Georgia, serif')
        df_plot['label'] = df_plot[col].apply(lambda v: f"{v:,.1f}".replace(',', 'X').replace('.', ',').replace('X', '.'))

        covid_start = df_plot[df_plot['periodo_dt'].dt.year == 2020]
        covid_marker = covid_start['periodo_dt'].iloc[0] if not covid_start.empty else None

        tooltip_template = (
            "<span style='background-color:#ffffff;border:1px solid #ffffff;"
            "padding:6px 10px;display:inline-block;color:#1e1b18;'>"
            "<b>%{x|%Y %b}</b><br>"
            "Tasa informal no agro (%): %{y:.1f}%"
            "</span><extra></extra>"
        )

        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                x=df_plot['periodo_dt'],
                y=df_plot[col],
                mode='lines+markers',
                line=dict(color=neutral_color, width=3),
                marker=dict(color=neutral_color, size=8, line=dict(color='#ffffff', width=1.2)),
                hovertemplate=tooltip_template,
                showlegend=False
            )
        )

        if highlight_color and not df_plot.empty:
            max_row = df_plot.loc[df_plot[col].idxmax()]
            fig.add_trace(
                go.Scatter(
                    x=[max_row['periodo_dt']],
                    y=[max_row[col]],
                    mode='markers',
                    marker=dict(color=highlight_color, size=12, symbol='diamond', line=dict(color='#ffffff', width=1.5)),
                    hovertemplate='<b>Pico</b><br>'+label+': %{y:.1f}%<extra></extra>',
                    showlegend=False
                )
            )

        if covid_marker is not None:
            fig.add_vline(x=covid_marker, line=dict(color='#e11d48', width=2, dash='dash'))
            fig.add_annotation(
                x=covid_marker,
                y=df_plot[col].max(),
                text='<b>COVID-19</b>',
                showarrow=False,
                font=dict(family=font_family, size=10, color='#e11d48'),
                bgcolor='rgba(255,255,255,0.85)',
                bordercolor='#e11d48',
                borderwidth=1
            )

        fig.update_layout(
            template='plotly_white',
            title=dict(
                text='<b>Línea temporal: Tasa informal no agro (2017-2024)</b>',
                font=dict(family=font_family, size=24, color=text_color),
                x=0.05,
                y=0.95
            ),
            font=dict(family=font_family, size=15, color=text_color),
            plot_bgcolor=background_color,
            paper_bgcolor=background_color,
            margin=dict(l=120, r=120, t=110, b=100),
            hoverlabel=dict(bgcolor='rgba(0,0,0,0)', bordercolor='rgba(0,0,0,0)', font=dict(family=font_family, color=text_color)),
            hovermode='x',
            spikedistance=-1,
            hoverdistance=50,
            width=1200,
            height=620
        )

        fig.update_xaxes(
            title=dict(text='Trimestre móvil', font=dict(family=font_family, size=14, color=text_color)),
            showgrid=True,
            gridcolor=grid_color,
            griddash='dot',
            zeroline=False,
            tickformat='%Y',
            tickfont=dict(family=font_family, size=12, color=text_color),
            showspikes=True,
            spikemode='across',
            spikecolor=spike_color,
            spikethickness=1.2,
            spikedash='solid',
            spikesnap='cursor',
            hoverformat=' '
        )
        fig.update_yaxes(
            title=dict(text='Porcentaje', font=dict(family=font_family, size=14, color=text_color)),
            showgrid=True,
            gridcolor=grid_color,
            griddash='dot',
            zeroline=False,
            tickfont=dict(family=font_family, size=12, color=text_color),
            showspikes=False
        )

        fig.update_layout(
            annotations=[
                dict(
                    text='Los Ríos, ENE 2017 - MAY 2024',
                    x=0,
                    y=1.05,
                    xref='paper',
                    yref='paper',
                    showarrow=False,
                    font=dict(family=font_family, size=14, color='#4a4338')
                ),
                dict(
                    text='Fuente: Encuesta Nacional de Empleo (INE)',
                    x=0,
                    y=-0.18,
                    xref='paper',
                    yref='paper',
                    showarrow=False,
                    font=dict(family=font_family, size=11, color='#6e6252')
                )
            ]
        )

        fig.show()
        print(f'Línea temporal generada: {label}')

Línea temporal generada: Tasa informal no agro (%)


In [53]:
# Extraer información clave de los datos para actualizar el HTML

# Ver rango de trimestres disponibles
trimestres_unicos = sorted(merged['Trimestre Móvil'].unique())
print(f"Total trimestres disponibles: {len(trimestres_unicos)}")
print(f"Primer trimestre: {trimestres_unicos[0]}")
print(f"Último trimestre: {trimestres_unicos[-1]}")
print(f"Todos los trimestres: {trimestres_unicos}")

print("\n" + "="*50)

# Filtrar datos totales para cada indicador
for col in ['tasa_informal_sector', 'ocupados_informales', 'tasa_informal_noagro']:
    if col not in merged.columns:
        continue
    
    df_plot = merged[['Trimestre Móvil','DTI_CL_SEXO', col]].copy()
    if '_T' in df_plot['DTI_CL_SEXO'].unique():
        df_plot = df_plot[df_plot['DTI_CL_SEXO'] == '_T']
    df_plot = df_plot.drop_duplicates(subset=['Trimestre Móvil']).sort_values('Trimestre Móvil')
    
    print(f"\n=== {col} ===")
    print(f"Períodos: {len(df_plot)}")
    print(f"Rango: {df_plot['Trimestre Móvil'].iloc[0]} a {df_plot['Trimestre Móvil'].iloc[-1]}")
    
    # Mostrar datos en formato JS para copiar fácilmente
    trimestres_js = ', '.join([f'"{t}"' for t in df_plot['Trimestre Móvil'].tolist()])
    valores_js = ', '.join([str(v) for v in df_plot[col].tolist()])
    
    print(f"trimestres: [{trimestres_js}]")
    print(f"valores: [{valores_js}]")

Total trimestres disponibles: 81
Primer trimestre: 2017 ago-oct
Último trimestre: 2024 mar-may
Todos los trimestres: ['2017 ago-oct', '2017 jul-sep', '2017 nov-ene', '2017 oct-dic', '2017 sep-nov', '2018 abr-jun', '2018 ago-oct', '2018 dic-feb', '2018 ene-mar', '2018 feb-abr', '2018 jul-sep', '2018 jun-ago', '2018 mar-may', '2018 may-jul', '2018 nov-ene', '2018 oct-dic', '2018 sep-nov', '2019 abr-jun', '2019 ago-oct', '2019 dic-feb', '2019 ene-mar', '2019 feb-abr', '2019 jul-sep', '2019 jun-ago', '2019 mar-may', '2019 may-jul', '2019 nov-ene', '2019 oct-dic', '2019 sep-nov', '2020 abr-jun', '2020 ago-oct', '2020 dic-feb', '2020 ene-mar', '2020 feb-abr', '2020 jul-sep', '2020 jun-ago', '2020 mar-may', '2020 may-jul', '2020 nov-ene', '2020 oct-dic', '2020 sep-nov', '2021 abr-jun', '2021 ago-oct', '2021 dic-feb', '2021 ene-mar', '2021 feb-abr', '2021 jul-sep', '2021 jun-ago', '2021 mar-may', '2021 may-jul', '2021 nov-ene', '2021 oct-dic', '2021 sep-nov', '2022 abr-jun', '2022 ago-oct', '2

In [54]:
# Extraer datos específicos para actualizar HTML
import json

# Crear diccionario con los datos para JavaScript
datos_para_html = {}

# Preparar datos base
df_total = merged[merged['DTI_CL_SEXO'] == '_T'].copy()
df_total = df_total.sort_values('Trimestre Móvil')

# Lista de trimestres únicos
trimestres_unicos = df_total['Trimestre Móvil'].tolist()

# Para cada indicador
indicadores = {
    'tasa_informal_sector': 'tasaInformal',
    'ocupados_informales': 'ocupadosInformales', 
    'tasa_informal_noagro': 'tasaNoAgro'
}

for col_python, col_js in indicadores.items():
    if col_python in df_total.columns:
        valores = df_total[col_python].tolist()
        datos_para_html[col_js] = {
            'trimestres': trimestres_unicos,
            'valores': valores,
            'total_periodos': len(valores)
        }

# Mostrar datos para copiar al JavaScript
print("=== DATOS PARA ACTUALIZAR EL HTML ===")
print(f"Total de trimestres: {len(trimestres_unicos)}")
print(f"Primer trimestre: {trimestres_unicos[0]}")
print(f"Último trimestre: {trimestres_unicos[-1]}")

print("\n=== ARRAYS JAVASCRIPT ===")
print("// Trimestres comunes")
trimestres_js = json.dumps(trimestres_unicos)
print(f"const trimestres = {trimestres_js};")

print("\n// Datos por indicador")
for col_js, data in datos_para_html.items():
    valores_js = json.dumps(data['valores'])
    print(f"const {col_js} = {valores_js};")

print(f"\n// Total períodos: {len(trimestres_unicos)}")
print(f"// Rango: {trimestres_unicos[0]} a {trimestres_unicos[-1]}")

# Guardar también en variable global
datos_para_html_global = datos_para_html

=== DATOS PARA ACTUALIZAR EL HTML ===
Total de trimestres: 81
Primer trimestre: 2017 ago-oct
Último trimestre: 2024 mar-may

=== ARRAYS JAVASCRIPT ===
// Trimestres comunes
const trimestres = ["2017 ago-oct", "2017 jul-sep", "2017 nov-ene", "2017 oct-dic", "2017 sep-nov", "2018 abr-jun", "2018 ago-oct", "2018 dic-feb", "2018 ene-mar", "2018 feb-abr", "2018 jul-sep", "2018 jun-ago", "2018 mar-may", "2018 may-jul", "2018 nov-ene", "2018 oct-dic", "2018 sep-nov", "2019 abr-jun", "2019 ago-oct", "2019 dic-feb", "2019 ene-mar", "2019 feb-abr", "2019 jul-sep", "2019 jun-ago", "2019 mar-may", "2019 may-jul", "2019 nov-ene", "2019 oct-dic", "2019 sep-nov", "2020 abr-jun", "2020 ago-oct", "2020 dic-feb", "2020 ene-mar", "2020 feb-abr", "2020 jul-sep", "2020 jun-ago", "2020 mar-may", "2020 may-jul", "2020 nov-ene", "2020 oct-dic", "2020 sep-nov", "2021 abr-jun", "2021 ago-oct", "2021 dic-feb", "2021 ene-mar", "2021 feb-abr", "2021 jul-sep", "2021 jun-ago", "2021 mar-may", "2021 may-jul", "2021 n

## 1. Configuración del Entorno e Importación de Librerías
Objetivo: Definir librerías, silenciar warnings no críticos y ajustar opciones de visualización para inspección clara.

## 2. Definición de Rutas y Parámetros
Objetivo: Centralizar rutas y parámetros reproducibles (semilla, umbrales, carpetas de salida).

## 3. Carga de Datasets Limpios
Objetivo: Cargar a memoria todos los archivos limpios requeridos manteniéndolos en un contenedor único para iteraciones consistentes.

## 4. Inspección Rápida de Estructura
Objetivo: Confirmar estructura, tipos y carga efectiva.

## 5. Verificación de Calidad Post-Limpieza
Objetivo: Validar ausencia de problemas residuales (nulos, duplicados, claves inconsistentes).

## 6. Diccionario de Variables (Generado)
Objetivo: Construir un resumen estructurado de cada columna para documentación reproducible.

## 7. Unificación / Merge de Datasets
Objetivo: Construir una tabla integrada alineada por trimestre y sexo para facilitar análisis longitudinal y comparativo.

In [20]:
# Integrar indicadores claves en un solo DataFrame ordenado
rename_map = {
    'INF_TOSI_CHL14_clean.csv': 'tasa_informal_sector',
    'INF_OI_CHL14_clean.csv': 'ocupados_informales',
    'INF_NOAGRO_CHL14_clean.csv': 'tasa_informal_noagro'
}
renamed_min = {}
for fname, df in datasets.items():
    temp = df.copy()
    if 'Value' in temp.columns:
        temp = temp.rename(columns={'Value': rename_map[fname]})
    keep_cols = ['Trimestre Móvil', 'DTI_CL_SEXO', rename_map[fname]]
    renamed_min[fname] = temp[keep_cols]

merged = None
for fname, df in renamed_min.items():
    if merged is None:
        merged = df.copy()
    else:
        merged = merged.merge(df, on=['Trimestre Móvil', 'DTI_CL_SEXO'], how='outer')

if merged[['Trimestre Móvil', 'DTI_CL_SEXO']].duplicated().any():
    agg_map = {c: 'mean' for c in merged.columns if c not in ['Trimestre Móvil', 'DTI_CL_SEXO']}
    merged = merged.groupby(['Trimestre Móvil', 'DTI_CL_SEXO'], as_index=False).agg(agg_map)

merged = merged.sort_values(['Trimestre Móvil', 'DTI_CL_SEXO']).reset_index(drop=True)
print('Shape merged:', merged.shape)
print('Columnas:', list(merged.columns))
display(merged.head())

Shape merged: (243, 5)
Columnas: ['Trimestre Móvil', 'DTI_CL_SEXO', 'ocupados_informales', 'tasa_informal_sector', 'tasa_informal_noagro']


,Trimestre Móvil,DTI_CL_SEXO,ocupados_informales,tasa_informal_sector,tasa_informal_noagro
0,2017 ago-oct,F,29.610,20.068,37.607
1,2017 ago-oct,M,40.899,24.949,33.888
2,2017 ago-oct,_T,70.508,22.994,35.497
3,2017 jul-sep,F,28.803,18.815,35.973
4,2017 jul-sep,M,41.074,23.562,33.761


### 9.2 📊 Visualización 1: Tasa de Ocupación Informal (%)

**¿Qué mide este indicador?**
- Porcentaje de trabajadores ocupados en empleos informales respecto del total de ocupados
- Refleja la magnitud relativa de la informalidad laboral en la economía
- Indicador clave para evaluar la calidad del empleo

**Interpretación:**
- **Valores en porcentaje**: Expresado como porcentaje del total de ocupados
- **Tendencia temporal**: Permite observar evoluciones de la formalización laboral
- **Impacto COVID-19**: Marca la línea temporal del primer trimestre 2020
- **Línea de promedio**: Referencia histórica para contextualizar períodos específicos

**Color identificativo:** 🔴 Rojo (#dc2626)

### 9.3 👥 Visualización 2: Ocupados Informales (personas)

**¿Qué mide este indicador?**
- Número absoluto de personas ocupadas en empleos informales
- Muestra el volumen real de trabajadores en situación de informalidad
- Permite dimensionar el impacto demográfico de las políticas laborales

**Interpretación:**
- **Valores absolutos**: Expresado en cantidad de personas (ej: 60,000 personas)
- **Tendencia demográfica**: Refleja cambios en el volumen de empleo informal
- **Variaciones estacionales**: Pueden reflejar ciclos económicos y cambios estructurales
- **Impacto pandemia**: Esperamos ver variaciones significativas en períodos de crisis

**Color identificativo:** 🔵 Azul (#2563eb)

### 9.4 🌾 Visualización 3: Tasa Informal No Agropecuario (%)

**¿Qué mide este indicador?**
- Porcentaje de ocupación informal excluyendo el sector agropecuario
- Ofrece una vista más urbana/industrial de la informalidad laboral
- Elimina el sesgo del sector agrícola que tradicionalmente tiene mayor informalidad

**Interpretación:**
- **Exclusión sectorial**: No incluye trabajos agrícolas, ganaderos o forestales
- **Focus urbano**: Refleja principalmente la informalidad en sectores secundarios y terciarios
- **Comparabilidad**: Permite comparar con otros países que excluyen el sector primario
- **Políticas focalizadas**: Útil para políticas dirigidas a sectores no agrícolas

**Color identificativo:** 🟢 Verde (#16a34a)

In [ ]:
# Visualización 3 bis: Línea temporal tasa informal no agro (%)
col = 'tasa_informal_noagro'
label = 'Tasa informal no agro (%)'
highlight_color = '#d62828'
neutral_color = '#0a9396'
background_color = '#f7f2ea'
grid_color = '#d9d3c5'
text_color = '#1e1b18'
spike_color = 'rgba(120, 120, 120, 0.35)'

if col not in serie_base.columns:
    print(f'Omitido {col} (no existe)')
else:
    from unicodedata import normalize
    import math

    df_plot = serie_base[['Trimestre Móvil', 'DTI_CL_SEXO', col]].copy()
    if '_T' in df_plot['DTI_CL_SEXO'].unique():
        df_plot = df_plot[df_plot['DTI_CL_SEXO'] == '_T']
    df_plot = df_plot.drop_duplicates(subset=['Trimestre Móvil']).sort_values('Trimestre Móvil')

    month_map = {
        'ENE': 1, 'FEB': 2, 'MAR': 3, 'ABR': 4, 'MAY': 5, 'JUN': 6,
        'JUL': 7, 'AGO': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DIC': 12
    }

    def periodo_to_timestamp(periodo):
        if not isinstance(periodo, str):
            return pd.NaT
        parts = periodo.split(' ', 1)
        if len(parts) != 2:
            return pd.NaT
        year, meses = parts
        primer_mes = meses.split('-')[0].strip()
        clave = normalize('NFKD', primer_mes).encode('ascii', 'ignore').decode('ascii').upper()[:3]
        month = month_map.get(clave)
        if month is None:
            return pd.NaT
        return pd.Timestamp(int(year), month, 1)

    df_plot['periodo_dt'] = df_plot['Trimestre Móvil'].apply(periodo_to_timestamp)
    df_plot = df_plot.dropna(subset=['periodo_dt'])
    df_plot = df_plot.sort_values('periodo_dt')

    if df_plot.empty:
        print('Sin datos para linea temporal')
    else:
        font_family = globals().get('ECONOMIST_FONT', 'Georgia, serif')
        df_plot['label'] = df_plot[col].apply(lambda v: f"{v:,.1f}".replace(',', 'X').replace('.', ',').replace('X', '.'))

        covid_start = df_plot[df_plot['periodo_dt'].dt.year == 2020]
        covid_marker = covid_start['periodo_dt'].iloc[0] if not covid_start.empty else None

        tooltip_template = (
            "<span style='background-color:#ffffff;border:1px solid #ffffff;"
            "padding:6px 10px;display:inline-block;color:#1e1b18;'>"
            "<b>%{x|%Y %b}</b><br>"
            "Tasa informal no agro (%): %{y:.1f}%"
            "</span><extra></extra>"
        )

        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                x=df_plot['periodo_dt'],
                y=df_plot[col],
                mode='lines+markers',
                line=dict(color=neutral_color, width=3),
                marker=dict(color=neutral_color, size=8, line=dict(color='#ffffff', width=1.2)),
                hovertemplate=tooltip_template,
                showlegend=False
            )
        )

        if highlight_color and not df_plot.empty:
            max_row = df_plot.loc[df_plot[col].idxmax()]
            fig.add_trace(
                go.Scatter(
                    x=[max_row['periodo_dt']],
                    y=[max_row[col]],
                    mode='markers',
                    marker=dict(color=highlight_color, size=12, symbol='diamond', line=dict(color='#ffffff', width=1.5)),
                    hovertemplate='<b>Pico</b><br>'+label+': %{y:.1f}%<extra></extra>',
                    showlegend=False
                )
            )

        if covid_marker is not None:
            fig.add_vline(x=covid_marker, line=dict(color='#e11d48', width=2, dash='dash'))
            fig.add_annotation(
                x=covid_marker,
                y=df_plot[col].max(),
                text='<b>COVID-19</b>',
                showarrow=False,
                font=dict(family=font_family, size=10, color='#e11d48'),
                bgcolor='rgba(255,255,255,0.85)',
                bordercolor='#e11d48',
                borderwidth=1
            )

        fig.update_layout(
            template='plotly_white',
            title=dict(
                text='<b>Línea temporal: Tasa informal no agro (2017-2024)</b>',
                font=dict(family=font_family, size=24, color=text_color),
                x=0.05,
                y=0.95
            ),
            font=dict(family=font_family, size=15, color=text_color),
            plot_bgcolor=background_color,
            paper_bgcolor=background_color,
            margin=dict(l=120, r=120, t=110, b=100),
            hoverlabel=dict(bgcolor='rgba(0,0,0,0)', bordercolor='rgba(0,0,0,0)', font=dict(family=font_family, color=text_color)),
            hovermode='x',
            spikedistance=-1,
            hoverdistance=50,
            width=1200,
            height=620
        )

        fig.update_xaxes(
            title=dict(text='Trimestre móvil', font=dict(family=font_family, size=14, color=text_color)),
            showgrid=True,
            gridcolor=grid_color,
            griddash='dot',
            zeroline=False,
            tickformat='%Y',
            tickfont=dict(family=font_family, size=12, color=text_color),
            showspikes=True,
            spikemode='across',
            spikecolor=spike_color,
            spikethickness=1.2,
            spikedash='solid',
            spikesnap='cursor',
            hoverformat=' '
        )
        fig.update_yaxes(
            title=dict(text='Porcentaje', font=dict(family=font_family, size=14, color=text_color)),
            showgrid=True,
            gridcolor=grid_color,
            griddash='dot',
            zeroline=False,
            tickfont=dict(family=font_family, size=12, color=text_color),
            showspikes=False
        )

        fig.update_layout(
            annotations=[
                dict(
                    text='Los Ríos, ENE 2017 - MAY 2024',
                    x=0,
                    y=1.05,
                    xref='paper',
                    yref='paper',
                    showarrow=False,
                    font=dict(family=font_family, size=14, color='#4a4338')
                ),
                dict(
                    text='Fuente: Encuesta Nacional de Empleo (INE)',
                    x=0,
                    y=-0.18,
                    xref='paper',
                    yref='paper',
                    showarrow=False,
                    font=dict(family=font_family, size=11, color='#6e6252')
                )
            ]
        )

        fig.show()
        print(f'Línea temporal generada: {label}')

Línea temporal generada: Tasa informal no agro (%)


In [54]:
# Extraer datos específicos para actualizar HTML
import json

# Crear diccionario con los datos para JavaScript
datos_para_html = {}

# Preparar datos base
df_total = merged[merged['DTI_CL_SEXO'] == '_T'].copy()
df_total = df_total.sort_values('Trimestre Móvil')

# Lista de trimestres únicos
trimestres_unicos = df_total['Trimestre Móvil'].tolist()

# Para cada indicador
indicadores = {
    'tasa_informal_sector': 'tasaInformal',
    'ocupados_informales': 'ocupadosInformales', 
    'tasa_informal_noagro': 'tasaNoAgro'
}

for col_python, col_js in indicadores.items():
    if col_python in df_total.columns:
        valores = df_total[col_python].tolist()
        datos_para_html[col_js] = {
            'trimestres': trimestres_unicos,
            'valores': valores,
            'total_periodos': len(valores)
        }

# Mostrar datos para copiar al JavaScript
print("=== DATOS PARA ACTUALIZAR EL HTML ===")
print(f"Total de trimestres: {len(trimestres_unicos)}")
print(f"Primer trimestre: {trimestres_unicos[0]}")
print(f"Último trimestre: {trimestres_unicos[-1]}")

print("\n=== ARRAYS JAVASCRIPT ===")
print("// Trimestres comunes")
trimestres_js = json.dumps(trimestres_unicos)
print(f"const trimestres = {trimestres_js};")

print("\n// Datos por indicador")
for col_js, data in datos_para_html.items():
    valores_js = json.dumps(data['valores'])
    print(f"const {col_js} = {valores_js};")

print(f"\n// Total períodos: {len(trimestres_unicos)}")
print(f"// Rango: {trimestres_unicos[0]} a {trimestres_unicos[-1]}")

# Guardar también en variable global
datos_para_html_global = datos_para_html

=== DATOS PARA ACTUALIZAR EL HTML ===
Total de trimestres: 81
Primer trimestre: 2017 ago-oct
Último trimestre: 2024 mar-may

=== ARRAYS JAVASCRIPT ===
// Trimestres comunes
const trimestres = ["2017 ago-oct", "2017 jul-sep", "2017 nov-ene", "2017 oct-dic", "2017 sep-nov", "2018 abr-jun", "2018 ago-oct", "2018 dic-feb", "2018 ene-mar", "2018 feb-abr", "2018 jul-sep", "2018 jun-ago", "2018 mar-may", "2018 may-jul", "2018 nov-ene", "2018 oct-dic", "2018 sep-nov", "2019 abr-jun", "2019 ago-oct", "2019 dic-feb", "2019 ene-mar", "2019 feb-abr", "2019 jul-sep", "2019 jun-ago", "2019 mar-may", "2019 may-jul", "2019 nov-ene", "2019 oct-dic", "2019 sep-nov", "2020 abr-jun", "2020 ago-oct", "2020 dic-feb", "2020 ene-mar", "2020 feb-abr", "2020 jul-sep", "2020 jun-ago", "2020 mar-may", "2020 may-jul", "2020 nov-ene", "2020 oct-dic", "2020 sep-nov", "2021 abr-jun", "2021 ago-oct", "2021 dic-feb", "2021 ene-mar", "2021 feb-abr", "2021 jul-sep", "2021 jun-ago", "2021 mar-may", "2021 may-jul", "2021 n